# NBA Salary Prediction Based on Age of player

This notebook demonstrates how to build a simple machine learning model to predict the salary of an NBA player based on the number of years they have played in the league.

## Step 1: Import Libraries and Load the Data

In [ ]:
# import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
import altair as alt
import snowflake.snowpark.types as t
import snowflake.snowpark.functions as f

from snowflake.ml.modeling.linear_model import LinearRegression
from snowflake.ml.modeling.neural_network import MLPRegressor
import snowflake.ml.modeling.metrics as metrics
from snowflake.ml.modeling.preprocessing import StandardScaler

In [ ]:
# Load your dataset from Snowflake
from snowflake.snowpark.context import get_active_session
session = get_active_session()

data = session.table('SNOWPARK_PLAYGROUND.NBA.SALARIES')

# Preview the data
data.show()

## Step 2: Data Exploration and Visualization

In [ ]:
# schema
list(data.schema)

In [ ]:
from snowflake.snowpark import DataFrame
from snowflake.snowpark.functions import lit, col, count

#let's check for missing values
def null_columns(df: DataFrame):
    total_records = df.count()
    for column in df.columns:
        not_null_count = count(column)
        df = df.withColumn(column, lit(total_records - not_null_count))
    return df.limit(1)
null_columns(data)

In [ ]:
# Basic statistics
st.write(data.describe())


# Visualize the relationship between age and salary
sns.scatterplot(x='AGE', y='SALARY', data=data.to_pandas())
plt.xlabel('AGE')
plt.ylabel('SALARY')
plt.title('Age vs Salary')
plt.show()

In [ ]:
# drop NaNs
data = data.dropna()

# Check for missing values
null_columns(data)

## Step 3: Prepare the Data

In [ ]:
# Select the feature(s) and target variable
X = data[['AGE']]  # Independent variable
y = data['SALARY']         # Dependent variable

# Split the data into training and testing sets
data_train, data_test = data.random_split(weights=[0.8, 0.2], seed=0)

# cast so we don't get complaints
data_train=\
data_train.select([f.col("AGE").cast('double').as_('AGE'),f.col("SALARY").cast('double').as_('SALARY')])

data_test=\
data_test.select([f.col("AGE").cast('double').as_('AGE'),f.col("SALARY").cast('double').as_('SALARY')])

data_train.describe()
# data_test.describe()

## Step 4: Train the Model

In [ ]:
features=['AGE']
labels=['SALARY']

In [ ]:
# Create a linear regression model
model = LinearRegression(
    input_cols=features,
    label_cols=labels
)


# Train the model
model.fit(data_train)


# Output the linear reg. model coefficients (have to convert to sklearn for this)
model_local=model.to_sklearn()
print(f'Intercept: {model_local.intercept_}')
print(f'Coefficient: {model_local.coef_[0]}')

## Step 5: Evaluate the Model

In [ ]:
# Make predictions on the test set
pred = model.predict(data_test)

In [ ]:
# check out the result
pred

In [ ]:

# Evaluate the model
mse = metrics.mean_squared_error(df=pred,y_true_col_names=['SALARY'], y_pred_col_names=['OUTPUT_SALARY'])
r2=metrics.r2_score(df=pred,y_true_col_name='SALARY', y_pred_col_name='OUTPUT_SALARY')

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

pred=pred.sort(col('AGE').desc())

y_pred = np.array(pred.select('OUTPUT_SALARY').collect())
x_test= np.array(pred.select('AGE').collect())

# plt.scatter(x='AGE',y='OUTPUT_SALARY',data=pred.to_pandas(), color='red')
plt.scatter(x='AGE',y='SALARY',data=data_test.to_pandas(), color='blue')
plt.plot(x_test, y_pred, color='red', linewidth=2)
plt.xlabel('AGE')
plt.ylabel('SALARY')
plt.title('Age vs Salary')
plt.show()




## Step 6: Make Predictions

In [ ]:
# Predict the salary for a player with a certain age
player_age=25
age = session.create_dataframe([player_age], schema=['AGE'])
predicted_salary = model.predict(age)
predicted_salary

## Step 7. Fit a Neural Network Model

In [ ]:
# Scale the data (standardization)
scaler = StandardScaler(
    input_cols=['AGE'],
    output_cols=['AGE_SCALED']
)

scaler.fit(data_test)
data_train_scaled = scaler.transform(data_train)
data_test_scaled=scaler.transform(data_test)


# Create the MLPRegressor model
mlp = MLPRegressor(
    input_cols=['AGE_SCALED'],
    label_cols=labels,
    hidden_layer_sizes=(64, 64), activation='relu', solver='lbfgs', max_iter=500, random_state=1)

# Train the model
res= mlp.fit(data_train_scaled)

pred_mlp=mlp.predict(data_test_scaled)

## Step 8. Evaluate a NNM

In [ ]:
pred_mlp

In [ ]:
# pred_mlp=pred_mlp.sort(col('AGE_SCALED').desc())
data_test=data_test.sort(col('AGE').desc())
data_sorted=data.sort(col('AGE').desc())

pred_mlp=pred_mlp.sort(col('AGE_SCALED').desc())


mse_mlp = metrics.mean_squared_error(df=pred_mlp,y_true_col_names=['SALARY'], y_pred_col_names=['OUTPUT_SALARY'])
r2_mlp=metrics.r2_score(df=pred_mlp,y_true_col_name='SALARY', y_pred_col_name='OUTPUT_SALARY')

print(f'Mean Squared Error: {mse_mlp}')
print(f'R^2 Score: {r2_mlp}')

x_test= np.array(data_test.select('AGE').collect())
y_pred = np.array(pred_mlp.select('OUTPUT_SALARY').collect())

# Visualize the results
plt.scatter(x='AGE',y='SALARY',data=data_sorted.to_pandas(), color='blue', label='Actual')
# plt.scatter(X_test, y_test, color='blue', label='Actual')
plt.plot(x_test, y_pred, color='red', label='Predicted')
plt.xlabel('Age')
plt.ylabel('Salary')
plt.title('MLPRegressor: Age vs Salary')
plt.legend()
plt.show()


## Step 9. make predictions

In [ ]:
# Predict the salary for a player with a certain number of years played
age = session.create_dataframe([player_age], schema=['AGE'])

age_scaled = scaler.transform(age)

predicted_salary = mlp.predict(age_scaled)
predicted_salary



In [ ]:
-- model registry
create schema if not exists snowpark_playground.registry

In [ ]:
#create Registry object
from snowflake.ml.registry import Registry

reg = Registry(session=session, database_name="SNOWPARK_PLAYGROUND", schema_name="REGISTRY")


In [ ]:
mv = reg.log_model(model,
                   model_name="nba_salary_lin_reg",
                   # conda_dependencies=["scikit-learn"],
                   comment="A LinearRegression ML model that predicts NBA salaries",
                   metrics={"r2": r2},
                   # sample_input_data=data_train,
                   options = {'relax_version':False}
                  )

In [ ]:
#what did we get back?
mv

In [ ]:
reg.log_model(mlp,
             model_name="nba_salary_mlp",
             comment = "A MLPRegressor ML model for predicting NBA salaries",
             metrics={"r2":r2_mlp,"mse":mse_mlp},
            options = {'relax_version':False}
            )

In [ ]:
reg.show_models()

## Summary
This notebook provides a simple implementation of a linear regression model and MLPRegressor model to predict NBA salaries based on the age of the player. 
Depending on the available data, the model could be enhanced by incorporating additional features or by using more complex models.
